In [1]:
import numpy as np
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend
import pandas as pd
import tensorflow as tf
import random
import os
from keras import layers
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from tensorflow.keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense, Concatenate
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, Model

NUM_TOKEN = 5003
MAX_PRO_LEN = 64
MAX_TXT_LEN = 256
NO_EPO = 60
NO_BAT = 128
CLS = 5001
SEP = 5002
PAD_ID = 0

MACHINE_1_P = "./data/set1_machine.json"
HUMAN_1_P = "./data/set1_human.json"
MACHINE_2_P = "./data/set2_machine.json"
HUMAN_2_P = "./data/set2_human.json"
TEST_P = "./data/test.json"
RANDOM_SEED = 42
MACHINE_IND = 0
HUMAN_IND = 1
TEST_FRA = 0.01

class DomainData:
    """
    train_test_split, pad_sequence, PCA, class_weight
    """
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def t_t_spli(self, test_size, random_state):
        self.random_state = random_state
        self.train_x, self.test_x, self.train_y, self.test_y = train_test_split(self.x, self.y, test_size=test_size, stratify = self.y, random_state = random_state)
        self.train_x = self.train_x.reset_index(drop=True)
        self.train_y = self.train_y.reset_index(drop=True)
        self.test_x = self.test_x.reset_index(drop=True)
        self.test_y = self.test_y.reset_index(drop=True)
        
    def add_sep(self):
        self.x["prompt"] = self.x.apply(lambda x: [i+1 for i in x["prompt"]] , axis = 1)
        self.x["prompt"] = self.x.apply(lambda x: [CLS] + x["prompt"] , axis = 1)
        self.x["prompt"] = self.x.apply(lambda x: x["prompt"] + [SEP] if len(x["prompt"])<MAX_PRO_LEN else x["prompt"][:MAX_PRO_LEN-1] + [SEP], axis = 1)

        self.x["txt"] = self.x.apply(lambda x: [i+1 for i in x["txt"]] , axis = 1)
        self.x["txt"] = self.x.apply(lambda x: [CLS] + x["txt"] , axis = 1)
        self.x["txt"] = self.x.apply(lambda x: x["txt"] + [SEP] if len(x["prompt"])<MAX_TXT_LEN else x["txt"][:MAX_TXT_LEN-1] + [SEP], axis = 1)
        
    
    def add_padding(self, padding, prompt_len, txt_len):
        self.train_prompt = self.train_x["prompt"]
        self.train_txt = self.train_x["txt"]
        self.train_label = self.train_y.to_numpy()
        self.test_prompt = self.test_x["prompt"]
        self.test_txt = self.test_x["txt"]
        self.test_label = self.test_y.to_numpy()
        unique_classes = np.unique(self.train_label)
        class_weights = class_weight.compute_class_weight("balanced", classes=unique_classes, y=self.train_y)
        self.class_weights = dict(zip(unique_classes, class_weights))
        
        self.prompt_len = prompt_len
        self.txt_len = txt_len
        
        self.train_prompt = pad_sequences(self.train_prompt, padding=padding, maxlen=prompt_len, value=PAD_ID)
        self.train_txt = pad_sequences(self.train_txt, padding=padding, maxlen=txt_len, value=PAD_ID)
        self.test_prompt = pad_sequences(self.test_prompt, padding=padding, maxlen=prompt_len, value=PAD_ID)
        self.test_txt = pad_sequences(self.test_txt, padding=padding, maxlen=txt_len, value=PAD_ID)
        
        
    def down_sampling(self):
        mac_ind = self.train_y[self.train_y == MACHINE_IND].index.to_list()
        hum_ind = self.train_y[self.train_y == HUMAN_IND].index.to_list()
        lower = min(len(mac_ind), len(hum_ind))
        sel_lit = mac_ind[:lower] + hum_ind[:lower]
        self.train_x = self.train_x.iloc[sel_lit]
        self.train_y = self.train_y.iloc[sel_lit]
        random.shuffle(sel_lit)

    def over_sampling(self, upper_fra):
        mac_ind = self.train_y[self.train_y == MACHINE_IND].index.to_list()
        hum_ind = self.train_y[self.train_y == HUMAN_IND].index.to_list()
        lower = min(len(mac_ind), len(hum_ind))
        if lower == len(mac_ind):
            upper = int(lower*upper_fra) if lower*upper_fra < len(hum_ind) else len(hum_ind)
            major = hum_ind[:upper]
            minor = mac_ind[:lower]
    
        else:
            upper = int(lower*upper_fra) if lower*upper_fra < len(mac_ind) else len(mac_ind)
            major = mac_ind[:upper]
            minor = hum_ind[:lower]
        
        add_n = upper - lower
        oversampled = []
        while(len(oversampled) < add_n):
            oversampled.append(random.choice(mac_ind))
        sel_lit = major + minor + oversampled
        random.shuffle(sel_lit)
        
        self.train_x = self.train_x.iloc[sel_lit]
        self.train_y = self.train_y.iloc[sel_lit]
    
    def test_down(self, frac = 1):
        mac_ind = self.test_y[self.test_y == MACHINE_IND].index.to_list()
        hum_ind = self.test_y[self.test_y == HUMAN_IND].index.to_list()
        lower = min(len(mac_ind), len(hum_ind))
        if frac > 1:
            sel_lit = mac_ind[:lower] + hum_ind[:int(lower/frac)]
        else:
            sel_lit = mac_ind[:int(lower*frac)] + hum_ind[:lower]
        self.test_x = self.test_x.iloc[sel_lit]
        self.test_y = self.test_y.iloc[sel_lit]
        random.shuffle(sel_lit)




def f1_loss(y_true, y_pred):
    # Calculate precision and recall
    tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
    fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)))
    fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)))
    precision = tp / (tp + fp + backend.epsilon())
    recall = tp / (tp + fn + backend.epsilon())
    
    # Calculate F1 score
    f1_score = 2 * precision * recall / (precision + recall + backend.epsilon())
    
    # Return negative F1 score as the loss (to minimize it)
    return -f1_score



import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer, Dense
from tensorflow.keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense
import numpy as np

class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), 
             Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim, mask_zero=True)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience = 3)
random.seed(RANDOM_SEED)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if len(tf.config.list_physical_devices('GPU')):
    tf.config.list_physical_devices('GPU')
    print("Using GPU")


2023-04-24 10:21:48.883480: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-24 10:21:49.071009: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 10:21:49.878033: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1
Using GPU


2023-04-24 10:21:51.227615: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-24 10:21:51.352715: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-24 10:21:51.353335: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


## Possible Improvement:
- max_prompt/txt
- batch
- hyperparameters

## Domain 1

In [2]:
## _______________ Read data from domain 1 _______________
man_1_df = pd.read_json(HUMAN_1_P)
man_1_df["label"] = HUMAN_IND
mac_1_df = pd.read_json(MACHINE_1_P).drop("machine_id", axis = 1)
mac_1_df["label"] = MACHINE_IND
domain_1_df = pd.concat([man_1_df, mac_1_df])

domain_1 = DomainData(domain_1_df[["prompt", "txt"]], domain_1_df["label"])
domain_1.add_sep()
domain_1.t_t_spli(TEST_FRA, RANDOM_SEED)
domain_1.over_sampling(1.6)
domain_1.test_down()
domain_1.add_padding('post', MAX_PRO_LEN, MAX_TXT_LEN)

encoder = OneHotEncoder()
train_y = encoder.fit_transform(np.array(domain_1.train_label).reshape(-1, 1)).toarray()
test_y = encoder.fit_transform(np.array(domain_1.test_label).reshape(-1, 1)).toarray()


/tmp/ipykernel_226154/1117775470.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.x["prompt"] = self.x.apply(lambda x: [i+1 for i in x["prompt"]] , axis = 1)
/tmp/ipykernel_226154/1117775470.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.x["prompt"] = self.x.apply(lambda x: [CLS] + x["prompt"] , axis = 1)
/tmp/ipykernel_226154/1117775470.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from tensorflow.keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense, Concatenate
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, Model
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), 
             Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim, mask_zero=True)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

vocab_size = 5000  # Only consider the top 20k words
embed_dim = 128  # Embedding size for each token
num_heads = 6  # Number of attention heads
ff_dim = 64  # Hidden layer size in feed forward network inside transformer
epo_size = NO_EPO
batch_size = 32
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# define model
inputs_p = Input(shape=(MAX_PRO_LEN,))
embedding_layer = TokenAndPositionEmbedding(MAX_PRO_LEN, vocab_size, embed_dim)
x = embedding_layer(inputs_p)
x = transformer_block(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(20, activation="relu")(x)
x = Dropout(0.1)(x)


# define model
inputs_t = Input(shape=(MAX_TXT_LEN,))
embedding_layer = TokenAndPositionEmbedding(MAX_TXT_LEN, vocab_size, embed_dim)
y = embedding_layer(inputs_t)
y = transformer_block(y)
y = GlobalAveragePooling1D()(y)
y = Dropout(0.1)(y)
y = Dense(20, activation="relu")(y)
y = Dropout(0.1)(y)


# Concatenate outputs from prompt and text models
merged = Concatenate()([x, y])
merged = Dense(units=64, activation='relu')(merged)
merged = Dense(20, activation="relu")(merged)
outputs = Dense(units=1, activation='sigmoid')(merged)
trans_model_2 = Model(inputs=[inputs_p, inputs_t], outputs=outputs)

# Compile and train
trans_model_2.compile(optimizer=optimizer, loss=loss, metrics=[f1_loss])
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('trans_model.h5', monitor='val_loss', save_best_only=True)
trans_model_2.fit([domain_1.train_prompt, domain_1.train_txt], domain_1.train_label, epochs=epo_size, batch_size=batch_size, validation_split=0.2, callbacks = [callback, model_checkpoint])
# trans_model_2.fit([domain_1.train_prompt, domain_1.train_txt], domain_1.train_label, epochs=epo_size, batch_size=batch_size, class_weight = domain_1.class_weights, validation_split=0.2, callbacks = [callback, model_checkpoint])
print("Model Saved: trans_model.h5")

trans_model_2 = tf.keras.models.load_model("trans_model.h5", custom_objects={ 'f1_loss': f1_loss, 'TransformerBlock': TransformerBlock, 'TokenAndPositionEmbedding':TokenAndPositionEmbedding })
print("Model Loaded: trans_model.h5")
loss, f1 = trans_model_2.evaluate([domain_1.test_prompt, domain_1.test_txt], domain_1.test_label, verbose=False)
print("loss: ", loss)
trans_1_pre_rnn = trans_model_2.predict([domain_1.test_prompt, domain_1.test_txt])
trans_1_pre_rnn = np.round(trans_1_pre_rnn).flatten()
confusion = confusion_matrix(domain_1.test_label, trans_1_pre_rnn)
# trans_1_pre_rnn = [0 if i.flatten()[0] > i.flatten()[1] else 1 for i in trans_1_pre_rnn]
# confusion = confusion_matrix(domain_1.test_label, trans_1_pre_rnn)
print(confusion)
f1 = f1_score(domain_1.test_label, trans_1_pre_rnn)
print("f1-score: ", f1)

# 31/33 -> Adam
# 34/29 -> SGD
# 33/31 -> RMSprop
# 34/29 -> Adagrad


2023-04-24 03:33:10.933392: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-24 03:33:10.934029: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-24 03:33:10.934301: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-24 03:33:11.922726: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-24 03:33:11.923128: I tensorflow/compile

Epoch 1/60


/home/junhual1/.local/lib/python3.8/site-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-04-24 03:33:16.034886: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-24 03:33:16.104051: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fdd4803b420 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-24 03:33:16.104109: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2023-04-24 03:33:16.140353: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DI

278/278 [==============================] - 61s 195ms/step - loss: 0.3172 - f1_loss: -0.8702 - val_loss: 0.2486 - val_f1_loss: -0.8968
Epoch 2/60
278/278 [==============================] - 24s 85ms/step - loss: 0.1926 - f1_loss: -0.9216 - val_loss: 0.2571 - val_f1_loss: -0.8973
Epoch 3/60
278/278 [==============================] - 18s 63ms/step - loss: 0.1143 - f1_loss: -0.9582 - val_loss: 0.1669 - val_f1_loss: -0.9429
Epoch 4/60
278/278 [==============================] - 16s 58ms/step - loss: 0.0613 - f1_loss: -0.9774 - val_loss: 0.1682 - val_f1_loss: -0.9467
Epoch 5/60
278/278 [==============================] - 13s 46ms/step - loss: 0.0418 - f1_loss: -0.9859 - val_loss: 0.1330 - val_f1_loss: -0.9519
Epoch 6/60
278/278 [==============================] - 11s 39ms/step - loss: 0.0258 - f1_loss: -0.9919 - val_loss: 0.2720 - val_f1_loss: -0.9421
Epoch 7/60
278/278 [==============================] - 11s 41ms/step - loss: 0.0153 - f1_loss: -0.9951 - val_loss: 0.1908 - val_f1_loss: -0.9577
Ep

In [ ]:
# trans_model_2 = tf.keras.models.load_model("trans_model.h5", custom_objects={ 'f1_loss': f1_loss, 'TransformerBlock': TransformerBlock, 'TokenAndPositionEmbedding':TokenAndPositionEmbedding })
# print("Model Loaded: trans_model.h5")
# loss, f1 = trans_model_2.evaluate([domain_1.test_prompt, domain_1.test_txt], domain_1.test_label, verbose=False)
# print("loss: ", loss)
# trans_1_pre_rnn = trans_model_2.predict([domain_1.test_prompt, domain_1.test_txt])
# trans_1_pre_rnn = np.round(trans_1_pre_rnn).flatten()
# confusion = confusion_matrix(domain_1.test_label, trans_1_pre_rnn)
# # trans_1_pre_rnn = [0 if i.flatten()[0] > i.flatten()[1] else 1 for i in trans_1_pre_rnn]
# # confusion = confusion_matrix(domain_1.test_label, trans_1_pre_rnn)
# print(confusion)
# f1 = f1_score(domain_1.test_label, trans_1_pre_rnn)
# print("f1-score: ", f1)

## Domain 2 weighted

In [2]:
over_fra = 1.6
weight_fra = 300

## _______________ Read data from domain 1 _______________
man_1_df = pd.read_json(HUMAN_1_P)
man_1_df["label"] = HUMAN_IND
mac_1_df = pd.read_json(MACHINE_1_P).drop("machine_id", axis = 1)
mac_1_df["label"] = MACHINE_IND
domain_1_df = pd.concat([man_1_df, mac_1_df])

domain_1 = DomainData(domain_1_df[["prompt", "txt"]], domain_1_df["label"])
domain_1.add_sep()
domain_1.t_t_spli(TEST_FRA, RANDOM_SEED)
# domain_1.down_sampling()
domain_1.add_padding('post', MAX_PRO_LEN, MAX_TXT_LEN)

## _______________ Read data from domain 2 _______________
man_2_df = pd.read_json(HUMAN_2_P)
man_2_df["label"] = HUMAN_IND
mac_2_df = pd.read_json(MACHINE_2_P).drop("machine_id", axis = 1)
mac_2_df["label"] = MACHINE_IND
domain_2_df = pd.concat([man_2_df, mac_2_df])

domain_2 = DomainData(domain_2_df[["prompt", "txt"]], domain_2_df["label"])
domain_2.add_sep()
domain_2.t_t_spli(TEST_FRA, RANDOM_SEED)
# domain_2.over_sampling(over_fra)
domain_2.test_down()
domain_2.add_padding('post', MAX_PRO_LEN, MAX_TXT_LEN)



## _______________ weight data _______________
sample_weight_1 = np.ones(len(domain_1.train_label))
sample_weight_2 = np.ones(len(domain_2.train_label))
sample_weight_2 *= weight_fra
sample_weight = np.concatenate([sample_weight_1, sample_weight_2])

train_prompt = np.concatenate([domain_1.train_prompt, domain_2.train_prompt])
train_txt = np.concatenate([domain_1.train_txt, domain_2.train_txt])
train_label = np.concatenate([domain_1.train_label, domain_2.train_label])

data = list(zip(train_prompt, train_txt, train_label, sample_weight))
random.shuffle(data)

train_prompt, train_txt, train_label, sample_weight = zip(*data)
train_prompt = np.array(train_prompt)
train_txt = np.array(train_txt)
train_label = np.array(train_label)
sample_weight = np.array(sample_weight)

/tmp/ipykernel_236438/1117775470.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.x["prompt"] = self.x.apply(lambda x: [i+1 for i in x["prompt"]] , axis = 1)
/tmp/ipykernel_236438/1117775470.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.x["prompt"] = self.x.apply(lambda x: [CLS] + x["prompt"] , axis = 1)
/tmp/ipykernel_236438/1117775470.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from tensorflow.keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense, Concatenate
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, Model
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), 
             Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim, mask_zero=True)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

vocab_size = 5000  # Only consider the top 20k words
embed_dim = 128  # Embedding size for each token
num_heads = 8  # Number of attention heads
ff_dim = 20  # Hidden layer size in feed forward network inside transformer
epo_size = NO_EPO
batch_size = 128
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)

# define model
inputs_p = Input(shape=(MAX_PRO_LEN,))
embedding_layer = TokenAndPositionEmbedding(MAX_PRO_LEN, vocab_size, embed_dim)
x = embedding_layer(inputs_p)
x = transformer_block(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.5)(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.5)(x)


# define model
inputs_t = Input(shape=(MAX_TXT_LEN,))
embedding_layer = TokenAndPositionEmbedding(MAX_TXT_LEN, vocab_size, embed_dim)
y = embedding_layer(inputs_t)
y = transformer_block(y)
y = GlobalAveragePooling1D()(y)
y = Dropout(0.5)(y)
y = Dense(16, activation="relu")(y)
y = Dropout(0.5)(y)


# Concatenate outputs from prompt and text models
merged = Concatenate()([x, y])
merged = Dense(units=64, activation='relu')(merged)
merged = Dense(20, activation="relu")(merged)
outputs = Dense(units=1, activation='sigmoid')(merged)
trans_model_2 = Model(inputs=[inputs_p, inputs_t], outputs=outputs)

# Compile and train
trans_model_2.compile(optimizer="adam", loss="binary_crossentropy", metrics=[f1_loss])
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('trans_model_weighted.h5', monitor='val_loss', save_best_only=True)
trans_model_2.fit([train_prompt, train_txt], train_label, epochs=epo_size, batch_size=batch_size, sample_weight = sample_weight, validation_split=0.2, callbacks = [callback, model_checkpoint])
print("Model Saved: trans_model_weighted.h5")

trans_model_2 = tf.keras.models.load_model("trans_model_weighted.h5", custom_objects={ 'f1_loss': f1_loss, 'TransformerBlock': TransformerBlock, 'TokenAndPositionEmbedding':TokenAndPositionEmbedding })
print("Model Loaded: trans_model_weighted.h5")
loss, f1 = trans_model_2.evaluate([domain_2.test_prompt, domain_2.test_txt], domain_2.test_label, verbose=False)
print("loss: ", loss)
trans_2_pre_rnn = trans_model_2.predict([domain_2.test_prompt, domain_2.test_txt])
trans_2_pre_rnn = np.round(trans_2_pre_rnn).flatten()
confusion = confusion_matrix(domain_2.test_label, trans_2_pre_rnn)
print(confusion)
f1 = f1_score(domain_2.test_label, trans_2_pre_rnn)
print("f1-score: ", f1)



2023-04-24 03:49:43.504718: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-24 03:49:43.505385: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-24 03:49:43.506106: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-24 03:49:44.662775: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-24 03:49:44.663839: I tensorflow/compile

Epoch 1/60


2023-04-24 03:50:03.608722: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-24 03:50:03.609005: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-24 03:50:03.934291: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f592790af60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-24 03:50:03.934343: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2023-04-24 03:50:03.994070: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-24 03:50:04.309967: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424

784/784 [==============================] - 139s 168ms/step - loss: 1.1511 - f1_loss: -0.9493 - val_loss: 0.8675 - val_f1_loss: -0.9854
Epoch 2/60
784/784 [==============================] - 96s 122ms/step - loss: 0.6174 - f1_loss: -0.9807 - val_loss: 0.4454 - val_f1_loss: -0.9843
Epoch 3/60
784/784 [==============================] - 94s 119ms/step - loss: 0.4028 - f1_loss: -0.9826 - val_loss: 0.3136 - val_f1_loss: -0.9855
Epoch 4/60
784/784 [==============================] - 92s 118ms/step - loss: 0.2459 - f1_loss: -0.9860 - val_loss: 0.3402 - val_f1_loss: -0.9875
Epoch 5/60
784/784 [==============================] - 93s 118ms/step - loss: 0.1106 - f1_loss: -0.9878 - val_loss: 0.1517 - val_f1_loss: -0.9897
Epoch 6/60
784/784 [==============================] - 93s 119ms/step - loss: 0.0768 - f1_loss: -0.9895 - val_loss: 0.1715 - val_f1_loss: -0.9907
Epoch 7/60
784/784 [==============================] - 92s 118ms/step - loss: 0.0640 - f1_loss: -0.9910 - val_loss: 0.1559 - val_f1_loss: -0.

### test on test

In [4]:
DOMAIN_SPL = 600

test_df = pd.read_json(TEST_P)
test_prompt = pad_sequences(test_df["prompt"], padding="post", maxlen=MAX_PRO_LEN)
test_txt = pad_sequences(test_df["txt"], padding="post", maxlen=MAX_TXT_LEN)

model_1 = tf.keras.models.load_model("trans_model.h5", custom_objects={ 'f1_loss': f1_loss, 'TransformerBlock': TransformerBlock, 'TokenAndPositionEmbedding':TokenAndPositionEmbedding })
model_2 = tf.keras.models.load_model("trans_model_weighted.h5", custom_objects={ 'f1_loss': f1_loss, 'TransformerBlock': TransformerBlock, 'TokenAndPositionEmbedding':TokenAndPositionEmbedding })

pred = []
pred += model_1.predict([test_prompt[:DOMAIN_SPL], test_txt[:DOMAIN_SPL]]).tolist()
pred += model_2.predict([test_prompt[DOMAIN_SPL:], test_txt[DOMAIN_SPL:]]).tolist()
pred = [int(i) for i in np.round(pred).flatten()]


13/13 [==============================] - 0s 14ms/step


In [5]:
pred_df = pd.DataFrame(pred)
pred_df.columns = ["Predicted"]
pred_df.index.names = ['Id']

pred_df.to_csv("./data/result3_v7.csv")

## Compare difference

In [6]:
# ori: transformer--tunned
# v1: config tunning
# v2: add sep, test_fra: 0.2 -- 0.582
# v3: test_fra = 0.5, retrain domain 2
# v4: test_fra = 0.5, retrain domain 1
# v5: test_fra = 0.5, class weight
# v6: test_fra = 0.01

In [9]:
df1 = pd.read_csv("./data/result3_v2.csv")
df2 = pd.read_csv("./data/result3_v2.csv")

df1["pred_2"] = df2["Predicted"]
df1["compare"] = df1.apply(lambda x: 1 if int(x["pred_2"] != x["Predicted"]) else 0, axis=1)

print("Difference: ", sum(df1["compare"].values.tolist()))

print(sum(df1["Predicted"].to_numpy()))
print(sum(df1["pred_2"].to_numpy()))

print(df1["compare"].values)
dis_m = []
for i in range(len(df1["compare"].values)):
    if df1["compare"][i] != 0:
        dis_m.append(i)
print(dis_m)

Difference:  72
895
935
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0
 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0
 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 

In [ ]:
df1 = pd.read_csv("./data/result3_v2.csv")
df2 = pd.read_csv("./data/result3_v3.csv")

df1["pred_2"] = df2["Predicted"]
df1["compare"] = df1.apply(lambda x: 1 if int(x["pred_2"] != x["Predicted"]) else 0, axis=1)

print("Difference: ", sum(df1["compare"].values.tolist()))

print(sum(df1["Predicted"].to_numpy()))
print(sum(df1["pred_2"].to_numpy()))

print(df1["compare"].values)
dis_m = []
for i in range(len(df1["compare"].values)):
    if df1["compare"][i] != 0:
        dis_m.append(i)
print(dis_m)

Difference:  11
895
884
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 